This is a proof of concept for a password bank system that securely stores and retrieves user passwords using encryption. We will cryptography to show how the mathematical principles behind encryption work in practice.

# Encrypt ceasar cipher

In [1]:
# Spør brukeren om tekst og shift-verdi
text = input("Skriv inn teksten du vil kryptere: ")
shift = int(input("Oppgi shift-verdi (f.eks. 3): "))

# Enkel Caesar cipher-funksjon
def caesar_cipher(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            base = 'A' if char.isupper() else 'a'
            result += chr((ord(char) - ord(base) + shift) % 26 + ord(base))
        else:
            result += char
    return result

encrypted = caesar_cipher(text, shift)
print(f"Kryptert tekst: {encrypted}")


Kryptert tekst: khl



# Decrypt ceasar cipher

In [2]:
def caesar_decrypt(ciphertext, shift):
    decrypted = ""
    for char in ciphertext:
        if char.isalpha():
            base = 'A' if char.isupper() else 'a'
            decrypted += chr((ord(char) - ord(base) - shift) % 26 + ord(base))
        else:
            decrypted += char
    return decrypted


# Bruker output fra krypteringsdelen over
decrypted_text = caesar_decrypt(encrypted, shift)
print(f"Dekryptert tekst: {decrypted_text}")


Dekryptert tekst: hei

